<a href="https://colab.research.google.com/github/davidegizzi/chicago_crime_gizzi/blob/main/chicago_crimes_gizzi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

#task 1
df = pd.io.gbq.read_gbq('''
  select count(distinct year) as period FROM bigquery-public-data.chicago_crime.crime;
  ''', project_id='valid-logic-332507', dialect='standard')
df.head()

#answer: the dataset covers a period of 21 years

In [ ]:
#task 2
df = pd.io.gbq.read_gbq('''
  select year, count(*) as crimes FROM bigquery-public-data.chicago_crime.crime
  group by year
  order by count(*) desc;  
''', project_id='valid-logic-332507', dialect='standard')
df.head(21)

#answer: year 2002 had the largest number of crimes

In [ ]:
#task 3, question 1

#I calculate the share of crimes leading to an arrest as the ratio between the number of crimes leading to an arrest for a specific year and the total number of crimes of during that year

df = pd.io.gbq.read_gbq('''
  select year, countif(arrest = 1)/count(*) as share from (
  select year, case when arrest= True then 1 else 0 end as arrest, from bigquery-public-data.chicago_crime.crime
  ) group by year
  order by share desc;
  ''', project_id='valid-logic-332507', dialect='standard')
df.head(21)

#answer: year 2005 had the largest share of crimes that led to an arrest

In [ ]:
#task 3, question 2
df = pd.io.gbq.read_gbq('''
  select year, count(*) as crimes FROM bigquery-public-data.chicago_crime.crime
  group by year
  order by year desc;  
''', project_id='valid-logic-332507', dialect='standard')
df.head(21)

#answer: except for the periods 2001-2002 and 2015-2016, for which the trend of crimes had slightly increased, the general trend is of decreasing number of crimes from 2002 until 2021.

In [ ]:
#task 4, question 1
df = pd.io.gbq.read_gbq('''
  select primary_type, count(primary_type) as frequency from bigquery-public-data.chicago_crime.crime
  where year=2020
  group by primary_type
  order by frequency desc;
  ''', project_id='valid-logic-332507', dialect='standard')
df.head()

#answer: the top five crimes in 2020 were battery, theft, criminal damage, assault and deceptive practice

In [ ]:
#task 4, question 2
df = pd.io.gbq.read_gbq('''
  select primary_type, countif(arrest = 1)/count(*) as arrest_rate from (
  select primary_type, case when arrest= True then 1 else 0 end as arrest, from bigquery-public-data.chicago_crime.crime
  where year=2020
  )
  group by primary_type;
  ''', project_id='valid-logic-332507', dialect='standard')
df.head(33)

#answer: the highest arrest rate was almost 16.6% for battery, while the lowest was about 2% for deceptive practice


In [ ]:
#task 5
df = pd.io.gbq.read_gbq('''
  select year, countif(arrest = 1) as arrests_true from (
  select year, case when arrest= True then 1 else 0 end as arrest, from bigquery-public-data.chicago_crime.crime
  ) group by year
  order by arrests_true desc;
  ''', project_id='valid-logic-332507', dialect='standard')
df.head(21)

#answer: year 2004 had the largest number of crimes that led to an arrest

In [ ]:
#task 6, question 1
import matplotlib.pyplot as plt

df = pd.io.gbq.read_gbq('''
  select year, countif(arrest = 1)/count(*) as arrest_rate from (
  select year, case when arrest= True then 1 else 0 end as arrest, from bigquery-public-data.chicago_crime.crime
  ) group by year
  order by year desc;
  ''', project_id='valid-logic-332507', dialect='standard')

plt.plot(df.year,df.arrest_rate,color='blue')
plt.scatter(df.year,df.arrest_rate,color='red')
plt.xlabel('year')
plt.xticks(range(2001,2021,2))
plt.ylabel(r'$\rm{arrest\:rate}$')
plt.show()

#answer: 
#we can identify two periods where there have been a steep decrease in the arrest rate: 
#1) between 2014 and 2016 where the arrest rate went down from 28.9% to 19.7%
#2) between 2019 and 2021 where the arrest rate went down from 21.5% to 12%

In [ ]:
#task 6, question 2
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.io.gbq.read_gbq('''
  select year, primary_type, count(primary_type) as frequency, countif(arrest = true) as tot_arrests from bigquery-public-data.chicago_crime.crime
  where year in (2014,2016,2019,2021)
  group by year, primary_type;
  ''', project_id='valid-logic-332507', dialect='standard')

sns.barplot(x="primary_type",y="tot_arrests",hue="year",data=df)
plt.ylabel(r'$\rm{total\:arrests}$')
plt.xticks(rotation='vertical')
plt.xlabel('')
plt.legend(loc='upper right')
plt.show()
sns.barplot(x="primary_type",y="frequency",hue="year",data=df)
plt.ylabel(r'$\rm{number\:of\:crimes}$')
plt.xlabel('')
plt.xticks(rotation='vertical')
plt.legend(loc='upper right')
plt.show()

#answer: 
#the main driver of the change is on one side the reduction of the total number of crimes when 
# moving from 2014 to 2016 (and equivalently from 2019 to 2020, see bottom panel), as well as 
# the crucial reduction of total arrests for specific types of crimes. For example,  
# it is possible to clearly see that the number of arrests by narcotics had a sensitive decrease by a half in 2016.
# Correspondingly, arrests by narcotics, battery and theft decreased considerably in 2021.

In [ ]:
#task 7, question 1 
df = pd.io.gbq.read_gbq('''
  select year, countif(arrest = 1)/count(*) as arrest_rate from (
  select year, case when arrest= True then 1 else 0 end as arrest, from bigquery-public-data.chicago_crime.crime
  where primary_type= 'THEFT' and year in (2017,2018)
  )
  group by year;
  ''', project_id='valid-logic-332507', dialect='standard')
df.head()

#answer:
#the arrest rate for thefts during 2017 and 2018 was about 10.06% and 9.7% respectively. 
#Therefore, there was a decrease of about 3.43% (this answer question 2 IF QUESTION 2 refers to the decrease of arrests for thefts).

In [ ]:
#task 7, question 2 
df = pd.io.gbq.read_gbq('''
  select year, countif(arrest = 1)/count(*) as arrest_rate from (
  select year, case when arrest= True then 1 else 0 end as arrest, from bigquery-public-data.chicago_crime.crime
  where year in (2017,2018)
  )
  group by year;
  ''', project_id='valid-logic-332507', dialect='standard')
df.head()

#answer:
#IF the question is referring to the total (i.e., considering all crimes) arrest rate between 2017 and 2018, it did not decrease but rather increased by about 2.46%. 

In [ ]:
#task 7, question 3
import numpy as np
from scipy.stats import ttest_ind
import pandas as pd

df = pd.io.gbq.read_gbq('''
  select year, case when arrest= True then 1 else 0 end as arrest, from bigquery-public-data.chicago_crime.crime
  where year in (2017,2018);
  ''', project_id='valid-logic-332507', dialect='standard')

group1= np.array(df.arrest[df.year==2017])
group2= np.array(df.arrest[df.year==2018])

#I could not complete this part of the task, since from the previous two questions I do not understand whether by 'decrease in arrest rate' you are referring to a decrease in the total arrest rate between 2017 and 2018, or to a decrease for thefts only.
#As shown above, the decrease in the total arrest rate does not decrease between 2017 and 2018 (see also plot in task 6, question 1) 